In [1]:
# import os
# os.mkdir("going_moduler_exercise")

In [5]:
%%writefile going_moduler_exercise/get_data.py

# 1. get the data
import os
import requests
import zipfile
from pathlib import Path


URL = "https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip"
# Setup path to a data folder
data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'


# If the imgae folder doesn't exist, download it and prepare it
if image_path.is_dir():
    print(f'{image_path} directory already exists... skipping download')
else:
    print(f'{image_path} directory does not exist... creating one....')
    image_path.mkdir(parents=True, exist_ok=True)


# Download pizza, steak, and sushi data
with open(data_path/ 'pizza_steak_sushi.zip', 'wb') as f:
    request = requests.get(URL)
    print("Downloading pizza, steak and sushi data...")
    f.write(request.content)
    print("Download done.......")


# Unzip pizza, steak and sushi data
with zipfile.ZipFile(data_path / 'pizza_steak_sushi.zip', 'r') as zip_ref:
    print("Unzipping pizza, steak and sushi data")
    zip_ref.extractall(image_path)
    print("Extracted All....")

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Overwriting going_moduler_exercise/get_data.py


In [6]:
!python going_moduler_exercise/get_data.py

data/pizza_steak_sushi directory already exists... skipping download
Download done.......
Unzipping pizza, steak and sushi data
Extracted All....


In [7]:
%%writefile going_moduler_exercise/data_setup.py
"""
Contains functionality for creating Pytorch DataLoader's for Image classification data
"""
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
    """Creates training and testing DataLoaders.
    
    Takes in a training directory and testing directory path and turns them into
    PyTorch Datasets & then into PyTorch DataLoaders.
    
    Args:
        train_dir: Path to train data directory.
        test_dir: Path to test data directory.
        transform: torchvision transforms to perform on training and testing data.
        batch_size: Number of sample per batch in each of the DataLoaders.
        num_workers: An integar for number of workers per DataLoader.
    
    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
        Where class_names is a list of the target classes.
    
    Examples:
        train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir="path/to/train_dir",
            test_dir="path/to/test_dir",
            transform=some_transform,
            batch_size=32,
            num_workers=4)
    """

    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(root=train_dir, transform=transform, target_transform=None)
    test_data = datasets.ImageFolder(root=test_dir, transform=transform, target_transform=None)

    # Get the class names
    class_names = train_data.classes

    # Turn datasets into DataLoaders
    train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    
    return train_dataloader, test_dataloader, class_names


Writing going_moduler_exercise/data_setup.py


In [8]:
%%writefile going_moduler_exercise/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG Model. 
"""

import torch
from torch import nn

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture

    Replicates the TinyVGG architechture from the CNN explainer website in PyTorch
    The original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
        input_shape: An integar indicating number of input channels.
        hidden_units: An integar indicating number of hidden units between layers.
        output_shape: An integar indicating number of output units.
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units, 
                      kernel_size=3, 
                      stride=1, 
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*13*13, 
                      out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))

Writing going_moduler_exercise/model_builder.py


In [9]:
%%writefile going_moduler_exercise/engine.py
"""
Contains functions for training and testing a PyTorch model
"""

import torch
import torchmetrics
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               accuracy: torchmetrics.classification.accuracy.Accuracy,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch
    
    Turns a target PyTorch model to training model and then runs
    through all of the required training step (forward pass, 
    loss claculation, optimizer step).

    Args:
        model: A PyTorch model to be trained.
        dataloader: A DataLoader instance for the model to be trained on.
        loss_fn: A PyTorch loss function to be minimized.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        accuracy: A torchmetric module to calculate accuracy.
        device: A target device to compute on (i.e. "cuda" or "cpu")

    Returns:
        A tuple of training loss and training accuracy metrics.
        In the form (train_loss, train_accuracy). For example:

        (0.1112, 0.8743)
    """
    # Put the model in train mode.
    model.train()

    # Setup train loss and train accuracy
    train_loss, train_accuracy = 0, 0

    # Loop through the dataloader data batches
    for batch, (X,y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward Pass
        y_logits = model(X)

        # 2. Claculate loss and accumulate loss
        loss = loss_fn(y_logits, y)
        # print(f"loss: \n {loss}")
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric accross all batches
        y_pred = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
        accuracy_value = accuracy(y_pred, y)
        # print(f"accuracy: \n {accuracy}")
        train_accuracy += accuracy_value.item()

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_accuracy = train_accuracy / len(dataloader)

    return train_loss, train_accuracy
    

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module, 
              accuracy: torchmetrics.classification.accuracy.Accuracy,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch
    
    Turns a target PyTorch model to "eval" mode and then performs
    forward pass on testing dataset and also calculate testing loss
    and testing accuracy.

    Args:
        model: A PyTorch model to be tested.
        dataloader: A DataLoader instance for the model to be tested on.
        loss_fn: A PyTorch loss function to calculate loss on the test data.
        accuracy: A torchmetric module to calculate accuracy.
        device: A target device to compute on (i.e. "cuda" or "cpu")

    Returns:
        A tuple of testing loss and testing accuracy metrics.
        In the form (test_loss, test_accuracy). For example:

        (0.0112, 0.9343)
    """
    # Put the model in eval mode
    model.eval()

    # Setup the test loss and test accuracy
    test_loss, test_accuracy = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batchs
        for batch, (X,y) in enumerate(dataloader):
            # Send data to a target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            y_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(y_logits, y)
            test_loss += loss.item()

            # Calculate accumulate accuracy
            y_pred = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
            accuracy_value = accuracy(y_pred, y)
            test_accuracy += accuracy_value.item()
    
    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_accuracy = test_accuracy / len(dataloader)

    return test_loss, test_accuracy
    

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          loss_fn: torch.nn.Module, 
          optimizer: torch.optim.Optimizer,
          accuracy: torchmetrics.classification.accuracy.Accuracy,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    """Trains and test a PyTorch model

    Passes a target PyTorch model through train_step() and test_step()
    functions for a number of epochs. training and testing the model in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
        model: A PyTorch model to be tested.
        train_dataloader: A DataLoader instance for the model to be trained on.
        test_dataloader: A DataLoader instance for the model to be tested on.
        loss_fn: A PyTorch loss function to calculate loss on the test data.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        accuracy: A torchmetric module to calculate accuracy.
        epochs: An integar indicating how many epochs to train for.
        device: A target device to compute on (i.e. "cuda" or "cpu")

    Returns:
        A dictionary of training and testing loss as well as training and
        testing accuracy metrics. Each metric has a value in a list for 
        each epoch.
        In the form: {train_loss: [...],
                      train_acc: [...],
                      test_loss: [...],
                      test_acc: [...]} 
        For example if training for epochs=2: 
                     {train_loss: [2.0616, 1.0537],
                      train_acc: [0.3945, 0.3945],
                      test_loss: [1.2641, 1.5706],
                      test_acc: [0.3400, 0.2973]} 
  """
    # Create empty results dictionary
    results = {
                "train_loss": [], 
                "train_accuracy": [], 
                "test_loss": [], 
                "test_accuracy": []
              }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_accuracy = train_step(model=model, 
                                                dataloader=train_dataloader, 
                                                loss_fn=loss_fn, 
                                                optimizer=optimizer, 
                                                accuracy=accuracy, 
                                                device=device)
        test_loss, test_accuracy = test_step(model=model, 
                                             dataloader=test_dataloader, 
                                             loss_fn=loss_fn,
                                             accuracy=accuracy, 
                                             device=device)
        
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss: .4f} | "
            f"train_accuracy: {train_accuracy: .4f} | "
            f"test_loss: {test_loss: .4f} | "
            f"test_accuracy: {test_accuracy: .4f}"
        )

    return results
    

Writing going_moduler_exercise/engine.py


In [10]:
%%writefile going_moduler_exercise/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""

import torch
from pathlib import Path

def save_model(model: torch.nn.Module, 
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory
    
    Args:
        model: A target PyTorch model to save.
        target_dir: A directory for saving the model to.
        model_name: A filename for the saved model. Should include either ".pth" or ".pt" as the file extension.

    Example Usage:
        save_model(model=model_1, 
                   target_dir='models', 
                   model_name='tiny_vgg_model.pth')    
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pth' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(), f=model_save_path)


Writing going_moduler_exercise/utils.py


In [12]:
%%writefile going_moduler_exercise/train.py
"""
Trains a PyTorch image classification model using device agnostic code.
"""

import os
import argparse
import torch
from torchvision import transforms
from torchmetrics import Accuracy
from timeit import default_timer as timer

import data_setup, engine, model_builder, utils


# Create a parser
parser = argparse.ArgumentParser(description="Get some hyperparameters...")


# Get an arg for num_epochs
parser.add_argument('--num_epochs', 
                    default=10, 
                    type=int, 
                    help="the number of epoch to train for.")

# Get an arg for batch_size
parser.add_argument("--batch_size", 
                    default=32, 
                    type=int, 
                    help="number of samples per batch")

# Get an arg for hidden_units
parser.add_argument("--hidden_units", 
                    default=10, 
                    type=int, 
                    help="number of hidden units between layers")

# Get an arg for learning rate
parser.add_argument("--learning_rate", 
                    default=0.001, 
                    type=float, 
                    help="learning rate to use for model")

# Get our arguments from the parser
args = parser.parse_args()

NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate
print(f"[INFO] Training a model for {NUM_EPOCHS} epochs with batch size {BATCH_SIZE} using {HIDDEN_UNITS} hidden units and a learning rate of {LEARNING_RATE}")


# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create tranform
data_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

# Create DataLoaders with help of data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir, 
                                                                               test_dir=test_dir, 
                                                                               transform=data_transform, 
                                                                               batch_size=BATCH_SIZE, 
                                                                               num_workers=os.cpu_count())

# Create model with help of mode_builder.py
model = model_builder.TinyVGG(input_shape=3, hidden_units=HIDDEN_UNITS, output_shape=len(class_names)).to(device)

# Set Loss function, Optimizer, Accuracy
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
accuracy = Accuracy(task='multiclass', num_classes=len(class_names)).to(device)


# Start the timer
start_time = timer()

# Start training with help of engine.py
engine.train(model=model, 
             train_dataloader=train_dataloader, 
             test_dataloader=test_dataloader, 
             loss_fn=loss_fn, 
             optimizer=optimizer, 
             accuracy=accuracy, 
             epochs=NUM_EPOCHS, 
             device=device)

# End timer
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")


# Save the model with help of utils.py
utils.save_model(model=model, target_dir="models", model_name="tiny_vgg_model_v1.pth")


Writing going_moduler_exercise/train.py


In [13]:
!python going_moduler_exercise/train.py --num_epochs 20 --batch_size 64 --hidden_units 64 --learning_rate 0.003

[INFO] Training a model for 20 epochs with batch size 64 using 64 hidden units and a learning rate of 0.003
100%|███████████████████████████████████████████| 20/20 [00:22<00:00,  1.13s/it]
[INFO] Total training time: 22.709 seconds
[INFO] Saving model to: models/tiny_vgg_model_v1.pth


In [21]:
%%writefile going_moduler_exercise/predict.py
import torch
import torchvision
import argparse
import model_builder


# Creating a parser
parser = argparse.ArgumentParser()

# Get an image path
parser.add_argument("--image_path", help="target image path to predict on")

# Get a model path
parser.add_argument("--model_path",
                    default="models/tiny_vgg_model_v1.pth",
                    type=str, 
                    help="target model to use for prediction filepath")

args = parser.parse_args()

# Setup class names
class_names = ['pizza', 'steak', 'sushi']

# Setup device
device = "cuda" if torch.cuda.is_available() else 'cpu'


# Get the image path
IMG_PATH = args.image_path
print(f"[INFO] Predicting on {IMG_PATH}")


def load_model(model_path: str=args.model_path):
    '''Load the saved model state dictionary
    Args:
        filepath: the filepath of the saved model
    '''
    # Need to use same hyperparameters as asaved model
    model = model_builder.TinyVGG(input_shape=3, 
                                  hidden_units=64, 
                                  output_shape=3).to(device)

    print(f"[INFO] Loading model from: {model_path}")

    # Load the model state_dict()
    model.load_state_dict(torch.load(f=model_path))

    return model


def prediction_on_image(image_path=IMG_PATH, model_path=args.model_path):
    # Load the model
    model = load_model(model_path=model_path)

    # Load the image and turn it into torch.float32 (same type as model)
    image = torchvision.io.decode_image(str(image_path)).type(torch.float32)

    # Divide the image pixel values by 255 to get them between [0, 1]
    image = image / 255.

    # Transform the image
    transform = torchvision.transforms.Compose([torchvision.transforms.Resize(size=(64,64))])
    image = transform(image)

    # Predict on image
    model.eval()
    with torch.inference_mode():
        # Put the image into target device
        image = image.to(device)

        # Get pred logits
        pred_logits = model(image.unsqueeze(dim=0)) # make sure image has batch dimension (shape: [batch_size, color_channels, height, width])

        # Get pred probs and label
        pred_prob = torch.softmax(pred_logits, dim=1)
        pred_label = torch.argmax(pred_prob, dim=1)

        # Get the preb label class
        pred_label_class = class_names[pred_label]

    print(f"[INFO] Predicted Class: {pred_label_class} | Predicted class probability: {pred_prob.max():.3f}")


if __name__ == "__main__":
    prediction_on_image()







Overwriting going_moduler_exercise/predict.py


In [22]:
!python going_moduler_exercise/predict.py --image_path pizza_dad.jpeg

[INFO] Predicting on pizza_dad.jpeg
[INFO] Loading model from: models/tiny_vgg_model_v1.pth
/media/niloy/Extra/Extra ubuntu Dec 2024/torch_learning/exercises/going_moduler_exercise/predict.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an is